In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pyrsgis

In [0]:
######################### Import des libraries #######################

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from copy import deepcopy
import torch
import torch.nn as nn
import pandas as pd
import scipy.signal
import scipy.ndimage
import os
import numpy as np

Warning! matplotlib_scalebar library not found.


In [0]:
############################## Parameters ############################

stop = 1000
epochs = 7

In [0]:
######################### Import des données #########################

path = "/content/drive/My Drive/D4G/"

#for testing
"""MSI = ['1_MSI_Seredou_2015_tiled/S2A_20151203_MSI_1km2_1.tif']
test = ['1_Images_Seredou_2015_16bit_tiled/S2A_20151203_seredou_1km2_1.tif']
truth = ['1_Mask_Seredou_2015_tiled/GroundTruth_seredou_20151203_1km2_1.tif']"""

"MSI = ['1_MSI_Seredou_2015_tiled/S2A_20151203_MSI_1km2_1.tif']\ntest = ['1_Images_Seredou_2015_16bit_tiled/S2A_20151203_seredou_1km2_1.tif']\ntruth = ['1_Mask_Seredou_2015_tiled/GroundTruth_seredou_20151203_1km2_1.tif']"

In [0]:
from torch.utils.data import Dataset, DataLoader, sampler

class OurDataset_train(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, idx_train, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        #for prod
        test_raw = os.listdir(path + "Images")
        MSI_raw = os.listdir(path + "MSI")
        truth_raw = os.listdir(path + "Maskm")

        self.idx = idx_train

        self.test = {i:('Images/'+test_raw[i]) for i in idx_train}
        self.MSI = {i:('MSI/'+MSI_raw[i]) for i in idx_train}
        self.mask = {i:('Maskm/'+truth_raw[i]) for i in idx_train}
        
    def __len__(self):
        return len(self.idx)

    def __getitem__(self, el):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 
                mask: 
                PatientID: 
                SourceDataset	
        """
        #Read the rasters as array
        ds1,featuretest_MSI = raster.read(path + self.MSI[self.idx[el]], bands='all')
        ds2,featuretest = raster.read(path + self.test[self.idx[el]], bands='all')
        ds3,truth = raster.read(path + self.mask[self.idx[el]], bands='all')
        
        #On remplace les valeurs négatives par 0
        featuretest_MSI[featuretest_MSI < 0] = 0
        featuretest[featuretest < 0] = 0
        featuretest = np.append(featuretest, np.array([featuretest_MSI]),axis=0)
        
        preprocessed_truth = np.array([changeDimension(truth)])
        
        #On importe les données
        for i in range (11):
        
            featuretest[i] = featuretest[i]/featuretest[i].max()
            
            if i == 0:
                preprocessed_data = np.array([changeDimension(featuretest[i])])
            else:
                preprocessed_data = np.append(preprocessed_data, [changeDimension(featuretest[i])], axis = 0)
            
            #On calcule les moyennes des cases adjacentes - creation de 11 nouvelles colonnes
            #featuretest_avg = scipy.signal.convolve2d(featuretest[i],np.ones((5,5)),'same')/25
            #Pour un filtre médian
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[25,25]))], axis = 0)
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[11,11]))], axis = 0)
            #preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[5,5]))], axis = 0)
            #preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[3,3]))], axis = 0)
            #preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.ndimage.gaussian_filter(featuretest[i],1))], axis = 0)
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.ndimage.gaussian_laplace(featuretest[i],1))], axis = 0)
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.ndimage.sobel(featuretest[i],1))], axis = 0)
        
        data = torch.from_numpy(np.transpose(preprocessed_data)).type(torch.FloatTensor)
        mask = torch.from_numpy(preprocessed_truth).type(torch.LongTensor)
        sample = {'data': data, 'mask': torch.squeeze(mask)}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

class OurDataset_test(Dataset):
    """This dataset includes .... """
    
       
    def __init__(self, path, idx_test, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        #for prod
        test_raw = os.listdir(path + "Images")
        MSI_raw = os.listdir(path + "MSI")

        self.idx = idx_test

        self.test = {i:('Images/'+test_raw[i]) for i in idx_test}
        self.MSI = {i:('MSI/'+MSI_raw[i]) for i in idx_test}
        
    def __len__(self):
        return len(self.idx)

    def __getitem__(self, i):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 
                mask: 
                PatientID: 
                SourceDataset	
        """
        #Read the rasters as array
        ds1,featuretest_MSI = raster.read(path + MSI[self.idx[i]], bands='all')
        ds2,featuretest = raster.read(path + test[self.idx[i]], bands='all')

        #On remplace les valeurs négatives par 0
        featuretest_MSI[featuretest_MSI < 0] = 0
        featuretest[featuretest < 0] = 0
        featuretest = np.append(featuretest, np.array([featuretest_MSI]),axis=0)

        #On importe les données
        for i in range (11):
        
            featuretest[i] = featuretest[i]/featuretest[i].max()
            
            if i == 0:
                preprocessed_data = np.array([changeDimension(featuretest[i])])
            else:
                preprocessed_data = np.append(preprocessed_data, [changeDimension(featuretest[i])], axis = 0)
            
            #On calcule les moyennes des cases adjacentes - creation de 11 nouvelles colonnes
            #featuretest_avg = scipy.signal.convolve2d(featuretest[i],np.ones((5,5)),'same')/25
            #Pour un filtre médian
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[25,25]))], axis = 0)
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[11,11]))], axis = 0)
            #preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[5,5]))], axis = 0)
            #preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.signal.medfilt(featuretest[i],[3,3]))], axis = 0)
            #preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.ndimage.gaussian_filter(featuretest[i],1))], axis = 0)
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.ndimage.gaussian_laplace(featuretest[i],1))], axis = 0)
            preprocessed_data = np.append(preprocessed_data, [changeDimension(scipy.ndimage.sobel(featuretest[i],1))], axis = 0)
        
        data = torch.from_numpy(np.transpose(preprocessed_data)).type(torch.FloatTensor)
        sample = {'data': data}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
def train(model, train_loader, criterion, optimizer, n_epochs):
    """
    Method used to train a nn
    
    Args:
        model: (nn.Module) the neural network
        train_loader: (DataLoader) a DataLoader wrapping the dataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
        optimizer: (torch.optim) an optimization algorithm
        n_epochs: (int) number of epochs performed during training

    Returns:
        best_model: (nn.Module) the trained neural network
    """
    best_model = deepcopy(model)
    train_best_loss = np.inf

    batch_size = train_loader.batch_size
    n = 10

    n_batches = n//batch_size

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        for i, data in enumerate(train_loader):
            images, mask = data['data'], data['mask']
            scans = torch.flatten(mask)
            outputs = torch.reshape(model(images), (scans.shape[0],4))
            loss = criterion(outputs, scans)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            optimizer.zero_grad()
            print("Batch {}/{} loss : {} accuracy : {}".format(i+1, n_batches, str(total_loss/((i+1)*batch_size)),str(int(torch.sum(torch.argmax(outputs,1) == scans))/scans.shape[0])))

        mean_loss = total_loss / len(train_loader.dataset)
        print('Epoch %i: loss = %f' % (epoch, mean_loss))

        if mean_loss < train_best_loss:
            best_model = deepcopy(model)
            train_best_loss = mean_loss
    
    return best_model

def test(model, data_loader, criterion):
    """
    Method used to test a CNN
    
    Args:
        model: (nn.Module) the neural network
        data_loader: (DataLoader) a DataLoader wrapping a MRIDataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
    
    Returns:
        results_df: (DataFrame) the label predicted for on the slice level.
        results_metrics: (dict) a set of metrics
    """
    model.eval()
    columns = ["participant_id", "slice_id", "proba0", "proba1",
               "true_label", "predicted_label"]
    results_df = pd.DataFrame(columns=columns)
    total_loss = 0

    batch_size = data_loader.batch_size
    n = 10000

    n_batches = n//batch_size
    
    with torch.no_grad():
        for i, data in enumerate(data_loader, 0):
             images, mask = data['data'], data['mask']
             scans = torch.flatten(mask)
             outputs = torch.reshape(model(images), (scans.shape[0],4))
             loss = criterion(outputs, scans)
             total_loss += loss.item()
             print("Batch {}/{} loss : {}".format(i+1, n_batches, str(total_loss/((i+1)*batch_size))))
       #     probs = nn.Softmax(dim=1)(outputs)
        #    _, predicted = torch.max(outputs.data, 1)
             print("Accuracy : {}".format(str(int(torch.sum(torch.argmax(outputs,1) == scans))/scans.shape[0])))

    print("Final loss : {}".format(str(total_loss/ len(data_loader.dataset))))
           
    return 0


def compute_metrics(ground_truth, prediction):
    """Computes the accuracy, sensitivity, specificity and balanced accuracy"""
    metrics_dict = dict()
    metrics_dict['cMatrix'] = confusion_matrix(ground_truth, prediction)
    metrics_dict['pscore'] = precision_score(ground_truth, prediction,average='micro')
    metrics_dict['rscore'] = recall_score(ground_truth, prediction,average='micro')
    
    return metrics_dict

In [0]:
########################## Modèle ############################

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.lin1 = nn.Linear(55, 200)
        self.lin2 = nn.Linear(200, 200)
        self.lin3 = nn.Linear(200, 200)
        self.lin4 = nn.Linear(200, 4)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.lin1(x)
        x = self.tanh(x)
        x = self.lin2(x)
        x = self.tanh(x)
        x = self.lin3(x)
        x = self.tanh(x)
        x = self.lin4(x)
        return self.relu(x)

In [0]:
########################## Prétraitement ############################

batch_size = 1000

#Decoupage en train test 
idx = [i for i in range(len(os.listdir(path + "MSI")))]
idx_train, idx_val= train_test_split(idx, test_size = 0.2, random_state=42, shuffle=True)

database_train = OurDataset_train(path, idx_train)
dataloader_train = DataLoader(database_train, batch_size=batch_size, drop_last=True)

model = Model()
criterion = nn.MultiMarginLoss(p=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
n_epochs = 5

In [0]:
#################### Entrainement du modèle ##########################

train(model, dataloader_train, criterion, optimizer, n_epochs)
torch.save(model.state_dict(), "model")

In [0]:
############################### Statistiques ################################

#Predict for test data 

database_val = OurDataset_train(path, idx_val)
dataloader_val = DataLoader(database_val, batch_size=batch_size, drop_last=True)

results_metrics = test(model, dataloader_val, criterion)